In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:


import os
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Lambda, Input, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU,ThresholdedReLU
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam,Nadam,Adamax,TFOptimizer
import cv2


img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)


# Split validation data to optimize classifier during training
X_train, X_test, y_train, y_test = np.load("/kaggle/input/alphabet/data.npy",allow_pickle=True )



# Train and Validation Data

for i in range(X_train.shape[0]):
  a=cv2.resize(X_train[i],(img_rows,img_cols))

  np.append(X_train_scaled,a)
  y_train[i]-=1
X_train_scaled=np.asarray(X_train_scaled)

for i in range(X_test.shape[0]):
  a=cv2.resize(X_test[i],(img_rows,img_cols))

  np.append(X_test_scaled,a)
  y_test[i]-=1
X_test_scaled=np.asarray(X_test_scaled)

rgb_X_train = np.repeat(X_train_scaled[..., np.newaxis], 1, -1)
rgb_X_test = np.repeat(X_test_scaled[..., np.newaxis], 1, -1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train = rgb_X_train
X_test = rgb_X_test


# Convert into format acceptable by tensorflow
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Model Parameters
batch_size = 128
num_classes = y_test.shape[1]
epochs = 80

# Parameter for Image Data Augumentation
shift_fraction=0.015

# Create Model
fashion_model = Sequential()
fashion_model.add(Conv2D(32,kernel_size=(3, 3),activation='linear',input_shape=input_shape,padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.5))
fashion_model.add(Conv2D(64,(3, 3),activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.5))
fashion_model.add(Conv2D(128,(3, 3),activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(Dropout(0.5))
fashion_model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
fashion_model.add(Flatten())
fashion_model.add(Dense(128,activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(Dense(num_classes,activation='softmax'))
fashion_model.compile(loss=keras.losses.categorical_crossentropy,optimizer=Adamax(),metrics=['accuracy'])
fashion_model.summary()

In [ ]:
# Train the model
#fashion_train = fashion_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val))

gen = ImageDataGenerator(width_shift_range=shift_fraction,height_shift_range=shift_fraction,horizontal_flip=True)
batches = gen.flow(X_train, y_train, batch_size=batch_size)
val_batches = gen.flow(X_test, y_test, batch_size=batch_size)

fashion_train=fashion_model.fit_generator(batches, steps_per_epoch=X_train.shape[0]//batch_size, epochs=epochs,validation_data=val_batches, 
                                                   validation_steps=X_test.shape[0]//batch_size, use_multiprocessing=True)



In [ ]:
# Evaluate Model against test data and get the score
score = fashion_model.evaluate(X_test, y_test, verbose=1)

# Print Metrics
print (score)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from tensorflow.keras.optimizers import SGD
import time
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Convolution1D,MaxPooling1D, Convolution2D, MaxPooling2D, Input
# Define model architecture
inputs = Input(shape=(28,28,1)) 
x = inputs 
filtre32=Convolution2D(32,(3,3), activation='relu',padding='same')(x)
filtre64=Convolution2D(64,(3,3), activation='relu',padding='same')(filtre32)
dropout1=Dropout(0.25)(filtre64)
p= MaxPooling2D(pool_size=2, strides=2,padding='same')(dropout1)
f=Flatten()(p) 

couche=Dense(256,activation='relu')(f)
dropout=Dropout(0.5)(couche)
outputs=Dense(10, activation='softmax')(dropout) 
model = Model(inputs, outputs) 

model.summary()
lr=0.1 
batch_size=256 
epochs=50
sgd1=SGD(lr=lr) #stochastic gradient descent with a fixed learning rate
print(x_train_new.shape)
print(y_batches.shape)
print(rgb_X_test.shape)
print(Y_test.shape)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
tps1 = time.clock() #on regarde le temps que ça tourne
history =model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                   verbose=1,validation_data=(X_test, y_test)) 
tps2 = time.clock()

print('lr=',lr,'batch_size=',batch_size, 'epochs=',epochs) 
print('Temps d apprentissage',tps2 - tps1)


In [ ]:
# Evaluate Model against test data and get the score
score = model.evaluate(X_test, y_test, verbose=1)

# Print Metrics
print (score)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from kerastuner.applications import HyperResNet
from kerastuner.tuners import Hyperband
from kerastuner import HyperParameters

hypermodel2 = HyperResNet(input_shape=(28, 28, 1), classes=10)

hp = HyperParameters()
hp.Choice('learning_rate', values=[1e-3, 1e-4])
hp.Fixed('optimizer', value='adam')


tuner_hb = Hyperband(
            hypermodel2,
            hyperparameters=hp,
            tune_new_entries=False,
            max_epochs=50,
            objective='val_accuracy',
            seed=42,
            executions_per_trial=3
        )

tuner_hb.search(rgb_X_train, Y_train,epochs=2,batch_size=128, validation_split=0.1)

In [ ]:

# Import libraries and modules
import time
import numpy as np
np.random.seed(123)  # for reproducibility

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Convolution1D,MaxPooling1D, Convolution2D, MaxPooling2D, Input

def affiche(history): 
  # summarize history for accuracy 
  plt.plot(history.history['accuracy']) 
  plt.plot(history.history['val_accuracy']) 
  plt.title('model accuracy') 
  plt.ylabel('accuracy') 
  plt.xlabel('epoch') 
  plt.legend(['train', 'test'], loc='upper left') 
  plt.grid()
  plt.show() 
  # summarize history for loss 
  plt.plot(history.history['loss']) 
  plt.plot(history.history['val_loss']) 
  plt.title('model loss') 
  plt.ylabel('loss') 
  plt.xlabel('epoch') 
  plt.legend(['train', 'test'], loc='upper left') 
  plt.grid()
  plt.show()

In [ ]:
best_hps = tuner_hb.get_best_hyperparameters(num_trials = 1)[0]

model = tuner_hb.hypermodel.build(best_hps)
tps1 = time.clock()
history =model.fit(rgb_X_train, Y_train, epochs = 100, validation_data = (rgb_X_test, Y_test))

tps2 = time.clock()

affiche(history)  